In [ ]:
from sklearn.model_selection import train_test_split # splitting the dataset
from sklearn.naive_bayes import GaussianNB # classification algorithm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report, roc_auc_score # metrics
import numpy as np # data processing
import pandas as pd # data processing
import matplotlib.pyplot as plt # data plotting

In [ ]:
# df=pd.read_csv("Heart.csv")
df.head()

In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
print("//Independent features//")
print(X.head())
print("\n\n//Dependent feature//")
print(y.head())

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)

In [ ]:
# Naive Bayes
nb=GaussianNB()
nb.fit(X_train,y_train)
nb_pred=nb.predict(X_test)
nb_con_matrix=confusion_matrix(y_test,nb_pred)
nb_acc=accuracy_score(y_test,nb_pred)
print("Confusion Matrix",'\n',nb_con_matrix,"\n")
print("Accuracy of Naive Bayes : ",nb_acc*100,"\n")
print("Classification Report",'\n',classification_report(y_test,nb_pred))

In [ ]:
y_score = nb.predict_proba(X_test)[:,1]
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_score)

In [ ]:
plt.figure(figsize=(5,5),dpi=150)
plt.plot(false_positive_rate,true_positive_rate, color='red', label = "Naive Bayes")
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],linestyle='--')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

In [ ]:
print("Area Under the Receiver Operating Characteristic Curve :",roc_auc_score(y_test,y_score))